## Intro

> Setup

In [ ]:
import pandas as pd
import pathlib
import os
import arcpy
from utils import *
import numpy as np
pd.options.mode.copy_on_write = True

In [ ]:
# current working directory
local_path = pathlib.Path().absolute()
# set data path as a subfolder of the current working directory TravelDemandModel\2022\data\
data_dir = local_path.parents[0] / '2022/data'
# set workspace
arcpy.env.workspace = os.path.join(local_path, 'Workspace.gdb')
# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)
# Set the extent environment using a feature class
arcpy.env.extent = "OccupancyRate_Zones_TAU"

In [ ]:
# globals
final_schema = ['APN', 'Residential_Units', 'TouristAccommodation_Units', 'CommercialFloorArea_SqFt',
                'TAU_Type', 'VHR', 'Lodging_Occupancy_Rate', 'PrimaryResidence_Rate', 'SecondaryResidence_Rate',
                'HighIncome_Rate',	'MediumIncome_Rate', 'LowIncome_Rate', 'PersonsPerUnit', 'ScalingFactor',
                'BlockGroupID', 'TAZ', 'Jurisdiction',	'County', 'Zoning_ID']


> Get Data

In [ ]:
# Get TAZ data
taz_url = 'https://maps.trpa.org/server/rest/services/Transportation_Planning/MapServer/6'
sdf_taz = get_fs_data_spatial(taz_url)
#set spatial reference
sdf_taz.spatial.sr = sr

# Get Unit Data
units_url = 'https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2'
sdf_units = get_fs_data_spatial_query(units_url, "Year = 2022")
sdf_units.spatial.sr = sr

# Get Block Group Data
block_groups_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/27'
sdf_block_groups = get_fs_data_spatial(block_groups_url)
sdf_block_groups = sdf_block_groups.loc[(sdf_block_groups['YEAR'] == 2020) & (sdf_block_groups['GEOGRAPHY'] == 'Block Group')]
sdf_block_groups.spatial.sr = sr

# Get VHR Data
vhr_url = 'https://maps.trpa.org/server/rest/services/VHR/MapServer/0'
sdf_vhr = get_fs_data_spatial(vhr_url)
sdf_vhr.spatial.sr = sr
# filter vhr layer to active status
sdf_vhr = sdf_vhr.loc[sdf_vhr['Status'] == 'Active']

census_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/28'
df_census = get_fs_data(census_url)
df_census_2022 = df_census.loc[(df_census['year_sample'] == 2022) & (df_census['sample_level'] == 'block group')]

# Get the data - should be 18 campgrounds
campground_url = 'https://maps.trpa.org/server/rest/services/Recreation/MapServer/1'
sdf_campground =  get_fs_data_spatial_query(campground_url, "RECREATION_TYPE='Campground'")

# campground occupancy rate data
dfCamp = pd.read_csv(os.path.join(data_dir,'Campground_Visitation.csv'))
dfCamp_2022 = dfCamp.loc[dfCamp['Year'] == 2022]

# occupancy feature class as a spatial dataframe
sdfOcc = pd.DataFrame.spatial.from_featureclass("Tahoe_OccupancyRate_Zones")
sdfOcc.spatial.sr = sr

# get table from geodatabase
# occupancy_rate = pd.DataFrame.spatial.from_table("OccupancyRates")
occupancy_rate = pd.read_csv(os.path.join(data_dir, 'Occupancy_Rate_Export.csv'))

# Get School Enrollment data
school_url_table     = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/32'
df_school_enrollment = get_fs_data(school_url_table)

# Get School Enrollment data - spatial
school_url_spatial = 'https://maps.trpa.org/server/rest/services/Datadownloader_PlanningandJurisdictions/MapServer/14'
sdf_school         =  get_fs_data_spatial(school_url_spatial)
sdf_school.spatial.sr = sr

> Utils to do

> general spatial joins

In [ ]:
# spatial join sdf units to sdf taz and sdf block groups
sdf_units_taz = sdf_units.spatial.join(sdf_taz, how='left')
sdf_units_block_groups = sdf_units.spatial.join(sdf_block_groups, how='left')

# match APN in join df and set to the value in join df
sdf_units.loc[sdf_units['APN'].isin(sdf_units_taz['APN']), 'TAZ'] = sdf_units_taz['TAZ_right']
# Use GEOID or TRPAID?
sdf_units.loc[sdf_units['APN'].isin(sdf_units_block_groups['APN']), 'BLOCKGROUP'] = sdf_units_block_groups['GEOID']

In [ ]:
sdf_units.BLOCKGROUP.value_counts() 

In [ ]:
sdf_units.TAZ.value_counts()

## Occupancy Rates

> Filter Occupancy Rate table to Timeframe and Room Type, Merge with Occupancy Zone Feature Class, and Export to Feature Class

In [ ]:
# occupacny rate table filter by room type and timeframe, merge with occupancy rate zones, and export to feature class
# filter by room type and timeframe
# filter for date(2022, 8, 1, 0, 0), 'Q4 21-22', and  'Q3 2022'
dfOccupancyTAU = occupancy_rate.loc[occupancy_rate.Timeframe.isin(['Q4 21-22', 'Q3 2022','2022-08-01']) & ~occupancy_rate.RoomType.isin(['VHR'])]
dfOccupancyVHR = occupancy_rate.loc[occupancy_rate.Timeframe.isin(['Q4 21-22', 'Q3 2022','2022-08-01']) & occupancy_rate.RoomType.isin(['VHR'])]

# cast Zone_ID as type string
dfOccupancyVHR['Zone_ID'].astype(str)
dfOccupancyTAU['Zone_ID'].astype(str)

# merge occupancy rate data to occupancy zones
sdf = pd.merge(sdfOcc, dfOccupancyTAU, left_on='OccupancyRate_ZoneID', right_on='Zone_ID', how='left')
# export sdf to feature class
sdf.spatial.to_featureclass(location=os.path.join('Workspace.gdb', 'OccupancyRate_Zones_TAU'), overwrite=True)

# merge occupancy rate data to occupancy zones
sdf = pd.merge(sdfOcc, dfOccupancyVHR, left_on='OccupancyRate_ZoneID', right_on='Zone_ID', how='left')
# export sdf to feature class
sdf.spatial.to_featureclass(location=os.path.join('Workspace.gdb', 'OccupancyRate_Zones_VHR'), overwrite=True)

> Classify parcel TAU Type and VHR

In [ ]:
# columns to keep
columns_to_keep = ['APN', 'Residential_Units', 'TouristAccommodation_Units',
                    'CommercialFloorArea_SqFt', 'YEAR',
                    'JURISDICTION', 'COUNTY', 'OWNERSHIP_TYPE',
                    'EXISTING_LANDUSE', 'TAZ', 'BLOCKGROUP','TAU_Type','VHR',
                    'WITHIN_TRPA_BNDY', 'PARCEL_ACRES', 'PARCEL_SQFT', 'SHAPE']

# List of Parcels APN with TAU Types
tau_lookup = pd.read_csv('Lookup_Lists/lookup_tau_type.csv')

# merge parcel 2022 with parcel VHR
df = sdf_units.merge(sdf_vhr, on='APN', how='left', indicator=True)

# calculate VHR = Yes if VHR is in the parcel
df['VHR'] = 'No'
df.loc[df['_merge'] == 'both', 'VHR'] = 'Yes'

# filter parcels so only APNs in the lookup are included
dfTAU = df[df['APN'].isin(tau_lookup['APN'])]
# get TAU_Type from lookup
dfTAU['TAU_Type'] = dfTAU['APN'].map(tau_lookup.set_index('APN')['TAU_Type'])

# setup TAU_Type
df['TAU_Type'] = 'N/A'
# any row with ToursitAccommodation_Units > 0 and TAU_Type is null, set TAU_Type to 'HotelMotel'
df.loc[(df['TouristAccommodation_Units'] > 0) & (df['TAU_Type']=='N/A'), 'TAU_Type'] = 'HotelMotel'
# for the rows in df that match rows by APN in dfTAU set TAU_Type to the value in dfTAU
df.loc[df['APN'].isin(dfTAU['APN']), 'TAU_Type'] = dfTAU['TAU_Type']

# remove _x from column names
df.columns = df.columns.str.replace('_x', '')
# columns to keep
df = df[columns_to_keep]

df.TAU_Type.value_counts()

In [ ]:
df.info()

> Join TAU and VHR parcels to respective occupancy rate zones

In [ ]:
# filter rows where VHR = Yes
sdf_units_2022_vhr = sdf_units_2022.loc[sdf_units_2022['VHR'] == 'Yes']
# filter rows where TUA > 0
sdf_units_2022_tau = sdf_units_2022.loc[sdf_units_2022['TouristAccommodation_Units'] > 0]

# spatial join VHR to occupancy rate zones with VHR values
spjoin_vhr = arcpy.analysis.SpatialJoin(sdf_units_2022_vhr, "OccupancyRate_Zones_VHR", "OccupancyRate_Zones_VHR_Parcels", 
                                        "JOIN_ONE_TO_ONE", "KEEP_ALL", None, "INTERSECT", None, None)

# spatial join VHR to occupancy rate zones with VHR values
spjoin_tau = arcpy.analysis.SpatialJoin(sdf_units_2022_tau, "OccupancyRate_Zones_TAU", "OccupancyRate_Zones_TAU_Parcels", 
                                        "JOIN_ONE_TO_ONE", "KEEP_ALL", None, "INTERSECT", None, None)

# feature to point in memmory
vhr_points = arcpy.management.FeatureToPoint("OccupancyRate_Zones_VHR_Parcels", "in_memory/vhr_points", "INSIDE")

tau_points = arcpy.management.FeatureToPoint("OccupancyRate_Zones_TAU_Parcels", "in_memory/tau_points", "INSIDE")

> Generate Spatial Interpolated Occupancy Rate Surfaces

In [ ]:
# Set the extent environment using a feature class
arcpy.env.extent = "OccupancyRate_Zones_TAU"

# make feature layer from in memory points
arcpy.management.MakeFeatureLayer("in_memory/vhr_points", "vhr_points_lyr")
arcpy.management.MakeFeatureLayer("in_memory/tau_points", "tau_points_lyr")
# filter out rows where occupancy rate is null or 0
arcpy.management.SelectLayerByAttribute("vhr_points_lyr", "NEW_SELECTION", "report_occ_rate IS NOT NULL AND report_occ_rate > 0")
arcpy.management.SelectLayerByAttribute("tau_points_lyr", "NEW_SELECTION", "report_occ_rate IS NOT NULL AND report_occ_rate > 0")

# set the output cell size
cell_size = 30
# set the power parameter
power = 2
# set the search radius
search_radius = 10000

# set the output raster
out_raster = 'tau_occupancy_rate'
# run the IDW for TAUs
arcpy.sa.Idw("tau_points_lyr", 
             z_field='report_occ_rate', 
             cell_size=cell_size, 
             power=power, 
             search_radius=search_radius).save(out_raster)

# set the output raster
out_raster = 'vhr_occupancy_rate'
# run the IDW for VHRs
arcpy.sa.Idw("vhr_points_lyr", 
             z_field='report_occ_rate', 
             cell_size=cell_size, 
             power=power, 
             search_radius=search_radius).save(out_raster)

# raster to polygon
# set the output feature class
out_feature = 'tau_occupancy_rate_poly'
# run the raster to polygon for TAUs
arcpy.conversion.RasterToPolygon('tau_occupancy_rate', out_feature)

# set the output feature class
out_feature = 'vhr_occupancy_rate_poly'
# run the raster to polygon for VHRs
arcpy.conversion.RasterToPolygon('vhr_occupancy_rate', out_feature)


> Fill in parcel level missing occupancy rates with interpolated values

In [ ]:
# spatial join occupancy rate polygons to parcels with missing values
# get parcels with missing values
sdf_units_2022_vhr_missing = sdf_units_2022_vhr.loc[sdf_units_2022_vhr['OccupancyRate_ZoneID'].isnull()]
sdf_units_2022_tau_missing = sdf_units_2022_tau.loc[sdf_units_2022_tau['OccupancyRate_ZoneID'].isnull()]

# spatial join VHR to occupancy rate interpolation polygons
spjoin_vhr_missing = arcpy.analysis.SpatialJoin(sdf_units_2022_vhr_missing, "vhr_occupancy_rate_poly", "OccupancyRate_Zones_VHR_Parcels_Missing", 
                                        "JOIN_ONE_TO_ONE", "KEEP_ALL", None, "INTERSECT", None, None)
# spatial join TAU to occupancy rate interpolation polygons
spjoin_tau_missing = arcpy.analysis.SpatialJoin(sdf_units_2022_tau_missing, "tau_occupancy_rate_poly", "OccupancyRate_Zones_TAU_Parcels_Missing", 
                                        "JOIN_ONE_TO_ONE", "KEEP_ALL", None, "INTERSECT", None, None)

# merge to original data
sdf_units_2022_vhr = pd.concat([spjoin_vhr, spjoin_vhr_missing])
sdf_units_2022_tau = pd.concat([spjoin_tau, spjoin_tau_missing])

# fill in missing report_occ_rate values
sdf_units_2022_vhr['report_occ_rate_x'] = sdf_units_2022_vhr['report_occ_rate_y']
sdf_units_2022_tau['report_occ_rate_x'] = sdf_units_2022_tau['report_occ_rate_y']

In [ ]:
columns_to_keep = ['APN', 'Residential_Units', 'TouristAccommodation_Units',
                    'CommercialFloorArea_SqFt', 'YEAR',
                    'JURISDICTION', 'COUNTY', 'OWNERSHIP_TYPE',
                    'EXISTING_LANDUSE', 'TAZ', 'TAU_Type','VHR',
                    'WITHIN_TRPA_BNDY', 'PARCEL_ACRES', 'PARCEL_SQFT', 'SHAPE',
                    'report_occ_rate_x']

# remove _x from column names
sdf_units_2022_vhr.columns = sdf_units_2022_vhr.columns.str.replace('_x', '')

In [ ]:
# get the occupancy rate featuer class as spatial dataframe
sdf_occ_rate_tau = pd.DataFrame.spatial.from_featureclass("OccupancyRate_Zones_TAU_Parcels")
# get the occupancy rate featuer class as spatial dataframe
sdf_occ_rate_vhr = pd.DataFrame.spatial.from_featureclass("OccupancyRate_Zones_VHR_Parcels")

In [ ]:
# filter sdf_units_2022 to rows where TAU is null or 0
sdf_units_2022_no_tau = sdf_occ_rate_tau.loc[(sdf_occ_rate_tau['report_occ_rate'] == 0)|(sdf_occ_rate_tau['report_occ_rate'].isnull())]

# filter sdf_units_2022 to rows where VHR is null or 0
sdf_units_2022_no_vhr = sdf_occ_rate_vhr.loc[(sdf_occ_rate_vhr['VHR'] == 'Yes')&(sdf_occ_rate_vhr['report_occ_rate'] == 0)|(sdf_occ_rate_vhr['report_occ_rate'].isnull())]



In [ ]:
# spatial join to occupancy rate raster
sdf_units_2022_no_tau = pd.DataFrame.spatial.from_featureclass("OccupancyRate_Zones_TAU_Parcels")
sdf_units_2022_no_vhr = pd.DataFrame.spatial.from_featureclass("OccupancyRate_Zones_VHR_Parcels")

# merge occupancy rate data to occupancy zones


> Campground Occupancy

In [ ]:
# merge campground data with occupancy rate data on campground name
sdf_campground = sdf_campground.merge(dfCamp_2022, left_on='RECREATION_NAME', right_on='Campground', 
                                      how='left', indicator=True)

# keep only columns of interest
sdf_campground = sdf_campground[['RECREATION_NAME', 'Occupancy_Rate','SHAPE']]

# filter sdf_campground to only campgrounds with occupancy rate data
sdf_campground = sdf_campground[sdf_campground['Occupancy_Rate'].notnull()]

# IDW to get the occupancy rate for each campground
# set the output cell size
cell_size = 500
# set the power parameter
power = 2
# set the search radius
search_radius = 5000
# set the output raster
out_raster = 'campground_occupancy_rate'
# run the IDW
arcpy.sa.Idw(in_features=sdf_campground, 
             z_field='Occupancy_Rate', 
             cell_size=cell_size, 
             power=power, 
             search_radius=search_radius).save(out_raster)

# spatial join to campground points with NaN occupancy rate
sdf_campground_nan = sdf_campground[sdf_campground['Occupancy_Rate'].isnull()]
# spatial join to campground points with NaN occupancy rate


In [ ]:
# merge campground data with occupancy rate data on campground name
sdf_campground = sdf_campground.merge(dfCamp_2022, left_on='RECREATION_NAME', right_on='Campground', 
                                      how='left', indicator=True)

# spatial join TAZ data to campground data
arcpy.SpatialJoin_analysis(sdf_campground, sdf_taz, 'taz_campground', 
                           'JOIN_ONE_TO_ONE', 'KEEP_ALL', 
                           match_option='HAVE_THEIR_CENTER_IN')

# read in output of spatial join as sdf
sdf_campground_taz = pd.DataFrame.spatial.from_featureclass('taz_campground')

# get sites sold by multiplying the number of sites by the occupancy rate
sdf_campground_taz['SitesSold'] = sdf_campground_taz['Total_Sites'] * sdf_campground_taz['Occupancy_Rate']

# group by TAZ and sum of sites sold within TAZ
sdf_campground_taz_grouped = sdf_campground_taz.groupby('TAZ').agg(
                                                {'SitesSold': 'sum'}).reset_index()


## Overnight Visitation

## School Enrollment

In [ ]:
# set Type to Null
sdf_school['TYPE'] = None
# set SchoolType to 'elementary' if it contains 'elementary' or 'magnet' or 'academy'
sdf_school.loc[sdf_school['NAME'].str.contains('elementary', case=False), 'TYPE'] = 'Elementary School'
# set SchoolType to 'middle' if it contains 'middle'
sdf_school.loc[sdf_school['NAME'].str.contains('middle', case=False), 'TYPE'] = 'Middle School'
# set SchoolType to 'high' if it contains 'high'
sdf_school.loc[sdf_school['NAME'].str.contains('high', case=False), 'TYPE'] = 'High School'
# set SchoolType to 'college' if it contains 'college'
sdf_school.loc[sdf_school['NAME'].str.contains('college', case=False), 'TYPE'] = 'College'
# set SchoolType to 'other' if it it does not contain any of the above
sdf_school.loc[sdf_school['TYPE'].isnull(), 'TYPE'] = 'Elementary School'

In [ ]:
# spatial join TAZs to School points
sdf_school_taz = sdf_school.spatial.join(sdf_taz, how='inner')
# group by TYPE and sum of Enrollment within TAZ 
sdf_school_taz_grouped = sdf_school_taz.groupby(['TYPE', 'TAZ']).agg(
                                                {'ENROLLMENT': 'sum'}).reset_index()
# unstack by TYPE as columns and TAZ as a column
sdf_school_taz_grouped_pivot = sdf_school_taz_grouped.pivot(index='TAZ', 
                                                            columns='TYPE', 
                                                            values='ENROLLMENT').reset_index()
# merge to sdf_taz to get all tazs
sdf_taz_school = pd.merge(sdf_taz, sdf_school_taz_grouped_pivot, how='left', on='TAZ')

# drop SHAPE column
sdf_taz_school = sdf_taz_school.drop(columns='SHAPE')
# fill NA with 0 for all rows
sdf_taz_school = sdf_taz_school.fillna(0)
# cast all fields to int
sdf_taz_school = sdf_taz_school.astype(int)
# rename columns
sdf_taz_school.rename(columns={'Elementary School':'elementary_school_enrollment',
                               'Middle School':'middle_school_enrollment',
                               'High School':'high_school_enrollment',
                               'College':'college_enrollment'}, inplace=True)

# export to csv
sdf_taz_school.to_csv(os.path.join('SchoolEnrollment.csv'), index=False)

## Socio Econ

> 

## TAZ Summary

> Scaling Factor Data Engineering

> Aggregations

## Forecasts